# Modélisation

In [1]:
import pandas as pd
import numpy as np
import missingno as msno


In [2]:
df = pd.read_csv('tracks.csv')
df.release_date = pd.to_datetime(df.release_date)


In [ ]:
df_2000 = df[df.release_date > '2000']
df_2000.drop(['id'],axis=1,inplace=True)
df_2000.drop_duplicates(inplace=True)
df_2000['Year'] = pd.to_datetime(df_2000.release_date).dt.year
df_2000['month'] = pd.to_datetime(df_2000.release_date).dt.month



In [28]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder,PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split,learning_curve,cross_val_predict,cross_validate
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression,ElasticNet,SGDRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from xgboost.sklearn import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import BayesianRidge
from sklearn.kernel_ridge import KernelRidge
from catboost import CatBoostRegressor
R_algorithm = [LinearRegression(),ElasticNet(),SGDRegressor(),XGBRegressor(),SVR(),BayesianRidge(),KernelRidge(),CatBoostRegressor()]

In [32]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 166972 entries, 264091 to 200305
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   duration_ms       166972 non-null  int64  
 1   explicit          166972 non-null  int64  
 2   danceability      166972 non-null  float64
 3   energy            166972 non-null  float64
 4   key               166972 non-null  int64  
 5   loudness          166972 non-null  float64
 6   mode              166972 non-null  int64  
 7   speechiness       166972 non-null  float64
 8   acousticness      166972 non-null  float64
 9   instrumentalness  166972 non-null  float64
 10  liveness          166972 non-null  float64
 11  valence           166972 non-null  float64
 12  tempo             166972 non-null  float64
 13  time_signature    166972 non-null  int64  
 14  Year              166972 non-null  int64  
 15  target            166972 non-null  int64  
dtypes: float64(9), 

In [6]:
X = df_2000.drop(['popularity','name','artists','release_date','id_artists'], axis=1)
y = df_2000['popularity']

# Convert the list column to string column

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Preprocess the training data
num_cols = X.select_dtypes(include='number').columns.tolist()
cat_cols = X.select_dtypes(include='object').columns.tolist()

num_transformer = StandardScaler()
cat_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(transformers=[('num', num_transformer, num_cols),
                                                ('cat', cat_transformer, cat_cols)])



MAE: 13.284096741596448


In [ ]:
import os
import warnings
import sys
import mlflow.sklearn
import logging
from urllib.parse import urlparse

# MLFLOW

In [9]:

experiment_id = mlflow.create_experiment("Spotify_prediction")

In [13]:
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
eval_data = X_train
eval_data["target"] = y_train

In [26]:
for algorithm in R_algorithm :

    pipe = make_pipeline(preprocessor,PolynomialFeatures(),algorithm)
    pipe.fit(X_train,y_train)
    print("l'algorithme {} est terminé, enregistrement dans mlflow ").__format__(str(algorithm))
    with mlflow.start_run(experiment_id=experiment_id):

        model_info = mlflow.sklearn.log_model(pipe, "model")
        for k,v in pipe.named_steps[-1].get_params().items():
            mlflow.log_param(k,v)
        result = mlflow.evaluate(
            model_info.model_uri,
            eval_data,
            targets="target",
            model_type="regressor",
            evaluators=["default"],
            )

[Pipeline] . (step 1 of 3) Processing columntransformer, total=   0.1s
[Pipeline]  (step 2 of 3) Processing polynomialfeatures, total=   0.2s
[Pipeline] ........ (step 3 of 3) Processing elasticnet, total=   2.6s


2023/02/21 15:52:16 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2023/02/21 15:52:18 INFO mlflow.models.evaluation.default_evaluator: Shap explainer Permutation is used.
Permutation explainer: 2001it [08:41,  3.80it/s]                          
Unable to serialize underlying model using MLflow, will use SHAP serialization
